In [1]:
import logging
import os

import densenets.dense_correspondence_manipulation.utils.utils as utils
import torch
from densenets.dataset.spartan_dataset_masked import SpartanDataset
from densenets.evaluation.evaluation import DenseCorrespondenceEvaluation
from densenets.training.training import DenseCorrespondenceTraining

logging.basicConfig(level=logging.INFO)

## Load the configuration for training

In [2]:
config_filename = os.path.join(
    utils.getDenseCorrespondenceSourceDir(),
    'config',
    'dense_correspondence',
    'dataset',
    'composite',
    'caterpillar_upright.yaml',
)
config = utils.getDictFromYamlFilename(config_filename)

train_config_file = os.path.join(
    utils.getDenseCorrespondenceSourceDir(),
    'config',
    'dense_correspondence',
    'training',
    'training.yaml',
)

train_config = utils.getDictFromYamlFilename(train_config_file)
dataset = SpartanDataset(config=config)

logging_dir = "trained_models/tutorials"
num_iterations = 3500
d = 3  # the descriptor dimension
name = "caterpillar_%d" % (d)
train_config["training"]["logging_dir_name"] = name
train_config["training"]["logging_dir"] = logging_dir
train_config["dense_correspondence_network"]["descriptor_dimension"] = d
train_config["training"]["num_iterations"] = num_iterations

TRAIN = True
EVALUATE = True

Using SpartanDataset:
   - intrainmode
   - number of scenes11
   - total images:    2851


## Train the network

This should take about ~12-15 minutes with a GTX 1080 Ti

In [3]:
class Test(torch.nn.Module):
    def __init__(self):
        super(Test, self).__init__()
        self.layer = torch.nn.Linear(1, 10)

    def forward(self, x):
        return self.layer(x)

In [4]:
t = Test()

In [5]:
t.cuda()

Test(
  (layer): Linear(in_features=1, out_features=10, bias=True)
)

In [6]:
t(torch.rand([20, 1]).cuda())

tensor([[-0.9260, -0.3454,  0.0805, -0.6740, -0.1332,  0.4803, -0.4895,  0.7961,
         -0.6849,  0.6011],
        [-0.9161, -0.3965, -0.1115, -0.5004, -0.0239,  0.2983, -0.6255,  0.8813,
         -0.4915,  0.6820],
        [-0.9045, -0.4570, -0.3385, -0.2952,  0.1053,  0.0831, -0.7863,  0.9822,
         -0.2628,  0.7777],
        [-0.8988, -0.4863, -0.4484, -0.1958,  0.1678, -0.0210, -0.8641,  1.0310,
         -0.1521,  0.8240],
        [-0.9282, -0.3336,  0.1247, -0.7140, -0.1583,  0.5222, -0.4582,  0.7764,
         -0.7294,  0.5825],
        [-0.9338, -0.3048,  0.2328, -0.8117, -0.2198,  0.6246, -0.3816,  0.7284,
         -0.8383,  0.5370],
        [-0.9266, -0.3420,  0.0932, -0.6855, -0.1404,  0.4923, -0.4805,  0.7904,
         -0.6977,  0.5958],
        [-0.9105, -0.4256, -0.2204, -0.4019,  0.0381,  0.1950, -0.7026,  0.9297,
         -0.3817,  0.7279],
        [-0.9362, -0.2924,  0.2793, -0.8537, -0.2463,  0.6687, -0.3487,  0.7078,
         -0.8852,  0.5174],
        [-0.9063, -

In [7]:
# All of the saved data for this network will be located in the
# code/data/pdc/trained_models/tutorials/caterpillar_3 folder

if TRAIN:
    print(f"training descriptor of dimension {d}")
    train = DenseCorrespondenceTraining(dataset=dataset, config=train_config)
    train.run()
    print(f"finished training descriptor of dimension {d}")

INFO:root:Loading pose data for scene 2018-04-10-16-02-59
INFO:root:Loading pose data for scene 2018-04-10-16-04-10
INFO:root:Loading pose data for scene 2018-04-10-16-06-26
INFO:root:Loading pose data for scene 2018-04-10-16-12-21
INFO:root:Loading pose data for scene 2018-04-10-16-13-37


training descriptor of dimension 3


INFO:root:Loading pose data for scene 2018-04-10-16-16-15
INFO:root:Loading pose data for scene 2018-04-16-14-25-19
INFO:root:Loading pose data for scene 2018-04-16-14-40-25
INFO:root:Loading pose data for scene 2018-04-10-16-08-46
INFO:root:Loading pose data for scene 2018-04-16-14-44-53
INFO:root:Loading pose data for scene 2018-04-16-15-23-41
INFO:root:enabling domain randomization
INFO:root:setting up tensorboard_logger
INFO:root:tensorboard logger started


using SINGLE_OBJECT_WITHIN_SCENE
logging_dir:/home/stannis/Downloads/dense_correspondence_data/pdc/trained_models/tutorials/caterpillar_3


/home/stannis/ltr/sandbox/stannis/densenets/densenets/dense_correspondence_manipulation/utils/utils.py:345: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  return (flat_pixel_locations % image_width, flat_pixel_locations // image_width)
/home/stannis/ltr/sandbox/stannis/densenets/densenets/dense_correspondence_manipulation/utils/utils.py:345: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor div

LOSS: 1.1997e+00
LOSS: 6.6009e-01
LOSS: 6.2890e-01
LOSS: 6.1507e-01
LOSS: 3.4687e-01
LOSS: 2.8737e-01


KeyboardInterrupt: 

## Evaluate the network quantitatively

This should take ~5 minutes.

In [ ]:
model_folder = os.path.join(logging_dir, name)
model_folder = utils.convert_data_relative_path_to_absolute_path(model_folder)

if EVALUATE:
    DCE = DenseCorrespondenceEvaluation
    num_image_pairs = 100
    DCE.run_evaluation_on_network(model_folder, num_image_pairs=num_image_pairs)

See `evaluation_quantitative_tutorial.ipynb` for a better place to display the plots.